In [1]:
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.1.2-alpha1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.0.0.

In [3]:
// Initialize Lets-Plot GeoTools extension. 
%use lets-plot-gt(gt="[23,)")

In [4]:
@file:DependsOn("org.geotools:gt-shapefile:[23,)")
@file:DependsOn("org.geotools:gt-cql:[23,)")
@file:DependsOn("org.geotools:gt-epsg-hsql:[23,)")

In [5]:
import org.geotools.feature.DefaultFeatureCollection
import org.geotools.feature.simple.SimpleFeatureBuilder
import org.geotools.feature.simple.SimpleFeatureTypeBuilder
import org.geotools.geometry.jts.JTS
import org.geotools.referencing.CRS

import org.locationtech.jts.geom.Coordinate
import org.locationtech.jts.geom.GeometryFactory
import org.locationtech.jts.geom.Polygon

In [6]:
val geomFactory = GeometryFactory()

val lon = -124.736342 to -66.945392
val lat = 24.521208 to 49.382808

val points = listOf(
    geomFactory.createPoint(Coordinate(lon.first, lat.first)),
    geomFactory.createPoint(Coordinate(lon.first, lat.second)),
    geomFactory.createPoint(Coordinate(lon.second, lat.second)),
    geomFactory.createPoint(Coordinate(lon.second, lat.first)),
    geomFactory.createPoint(Coordinate(lon.first, lat.first)),
)

In [7]:
val epsg4326 = CRS.decode("EPSG:4326", true)
val epsg5070 = CRS.decode("EPSG:5070", true)

val transform = CRS.findMathTransform(epsg4326, epsg5070)

In [8]:
fun getFeatureCollection(applyTransform: Boolean): DefaultFeatureCollection {
  
    val coordList: List<Coordinate> = points.map { point ->
        if (applyTransform) JTS.transform(point, transform).coordinate else point.coordinate 
    }
    val pg = geomFactory.createPolygon(coordList.toTypedArray())

    val builder = SimpleFeatureTypeBuilder()
    builder.name = "BBox"
    builder.crs = if (applyTransform) epsg5070 else epsg4326
    builder.add("geom", Polygon::class.java)

    val featureType = builder.buildFeatureType()
    val featureBuilder = SimpleFeatureBuilder(featureType)
    featureBuilder.add(pg)

    val featureCollection = DefaultFeatureCollection()
    featureCollection.add(featureBuilder.buildFeature(null))
    return featureCollection
}

In [9]:
// WGS84
val sd1 = getFeatureCollection(applyTransform = false).toSpatialDataset()

letsPlot() + geomMap(map = sd1, size = 0.7)

In [10]:
// Transform geometry with EPSG:5070 projection
val sd2 = getFeatureCollection(applyTransform = true).toSpatialDataset()

letsPlot() + geomMap(map = sd2, size = 0.7)

In [11]:
// Specify "provided" to keep the original CRS

letsPlot() + geomMap(map = sd2, size = 0.7, useCRS = "provided")